In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import lxml
from lxml import html
from lxml import etree
import numpy as np
import pandas as pd
import requests
import re
import csv
from csv import writer

from scipy import stats


In [2]:
#1. Daten die abgefragt werden dedfinieren
Ort = ["berlin", "frankfurt", "hamburg", "koeln", "leipzip", "muenchen", "stuttgart"]
Umkreis = 50
Objekt = ["haeuser"]
Seite = list(range(2))

In [3]:
#2. URL aufteilen
Website = "https://www.immowelt.de/liste/"
Slash = "/"
Snippet1 = "/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r="
Snippet2 = "&sd=DESC&sf=RELEVANCE&sp="

In [4]:
#3. Url zusammensetzen hier am Beispiel Ort[0]=Berlin, Objekt[0]=haeuser, und die erste Seite Seite[0]+1
# wird aufgerufen. Das +1 bei der Seite weil ja die Liste immer bei 0 anfängt aber die Seite 
#im Internet erst ab 1. Also 0+1 =1 
url = Website + Ort[0] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[0]+1)

print(url)

https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=1


In [5]:
#4. Nun kommt es zum eigentlichen WebScraping


#Url Aufrufen
page = requests.get(url)
 
# Parsen der Seite URL
tree = html.fromstring(page.content)
 
# Alle IDs der Seite URL in der Liste id Speichern. IDs sind immer eindeutig und nur einmal vergeben

id = tree.xpath(".//a/@id")
# printen der Liste um es zu überprüfen
for e in range(1):
    print(id)

['28auw54', '274he5l', '26ekx52', '26zkl56', '27drw5w', '28y9q52', '279kt5y', '27a225p', '27sge5l', '27w5v5j', '27jx85v', '27bge5l', '27nhe5l', '28xyz53', '26eq85w', '26pew5r', '28yzs53', '27yc95r', '26faw5f', '272pd5t']


In [6]:
#5. Auf dem Webscraping in 4. aufbauen
# xpath ist immer gleich aufgebaut beispiel an MaxOrt: = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')
# //div[@class]= er sucht alle div mit class (Klassenname)
# //div[@class="estateFacts-f11b0"] hier sucht er alle div mit unterstruktur class die estateFacts-f11b0 heißt
#von diesem Ort geht man in die Unterstruktur rein und zwar in das 1 div (div[1]) und dann in die 
#unterstrukur dieses ersten div dort geht man in span rein und mit /text zieht man den Text aus span raus 
#und speichert ihn in der Liste MaxOrt 

#6. das gleiche macht man identisch für alle Werte die man auslesen möchte 
# z.b prices hier muss man nicht mal in die Unterstrukturen rein sondern kann direkt aus der 
# div[@data-test="price"]/text() den Preistext rausziehen diesmal hat der Befehl nicht nach class gesucht sondern 
#nach data-test in div 

prices = tree.xpath('//div[@data-test="price"]/text()')
MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')
area = tree.xpath('//div[@data-test="area"]/text()')
rooms = tree.xpath('//div[@data-test="rooms"]/text()')

# Printen aller Ergebnisse die man aus diser 1. Seite rausgelesen hat
for e in range(1):
    print(prices)
for e in range(1):
    print(MaxOrt)
for e in range(1):
    print(area)
for e in range(1):
    print(rooms)


['2.130.000 €', '690.000 €', '698.000 €', '429.000 €', '548.000 €', '490.000 €', '1.700.000 €', '7.500.000 €', '4.490.000 €', '795.000 €', '699.500 €', '1.100.000 €', '1.140.000 €', '549.000 €', '950.000 €', '1.590.000 €', '760.000 €', '344.000 €', '2.950.000 €', '550.000 €']
['max. 20 km | Hoppegarten (Dahlwitz-Hoppegarten)', 'max. 10 km | Berlin / Lankwitz (Steglitz)', 'max. 10 km | Berlin (Weißensee)', 'max. 10 km | Berlin (Weißensee)', 'max. 10 km | Berlin (Buckow)', 'max. 10 km | Berlin (Lankwitz)', 'max. 10 km | Berlin (Lichterfelde)', 'max. 10 km | Berlin / Schmargendorf (Schmargendorf)', 'max. 10 km | Berlin (Lichterfelde)', 'max. 15 km | Berlin / Biesdorf (Biesdorf)', 'max. 15 km | Berlin (Französisch Buchholz)', 'max. 15 km | Berlin (Lichterfelde)', 'max. 15 km | Berlin (Lichterfelde)', 'max. 15 km | Berlin (Kaulsdorf)', 'max. 15 km | Berlin (Rudow)', 'max. 15 km | Berlin (Köpenick)', 'max. 15 km | Berlin / Tegel (Tegel)', 'max. 15 km | Puchanstraße 33, Berlin (Köpenick)', 'm

In [7]:
#7. Man erkennt dass bei MaxOrt nicht nur der max. Ort drinsteht sondern auch mehr
# im ersten Fall (erstes Objekt auf der ersten Seite zur URL) "'max. 20 km | Hoppegarten (Dahlwitz-Hoppegarten)"  
# wenn man nur die max. 20 km Abstand zum Ort haben will muss man also den Rest rauslöschen
#da die Seite immer gleich aufgebaut ist kann man dies gut machen 
#dafür geht man von der ersten Ziffer bis zum | und löscht den Rest raus mit dem Befehl
#MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")-1] die -1 steht dafür dass man noch eine 
#Ziffer mehr löscht von rechts nach links Gezeählt ab dem Zeichen "|"

#da man dies für jeden Wert in der Liste MaxOrt machen muss hier wieder eine For schleife mit range als Länge 
# der Liste
MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')

for e in range(len(MaxOrt)):
    MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")-1]
    
# für die Überprüfung wieder printen
for e in range(1):
    print(MaxOrt)


['max. 20 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km']


In [8]:
#8. Listen sind nicht gleich lang. 
# Beim Programmieren ist aufgefallen dass die Listen nicht immer gleich lang
# sind und somit können sie nicht ausgegeben werden 
# das ist der Fall da die Homepage nicht geflegt ist und manchmal keine Zimmeranzahl oder Fläche zum Objekt 
# angegeben ist. Nur die ID und der Preis sind für jedes Objekt "immer" angegeben
# dafür muss noch eine Lösung gefunden werden, dass man mit den Listen arbeiten kann
# hier sieht man es

prices = tree.xpath('//div[@data-test="price"]/text()')
area = tree.xpath('//div[@data-test="area"]/text()')
rooms = tree.xpath('//div[@data-test="rooms"]/text()')

#printen um zu überprüfen
for e in range(1):     
    print(id)
for e in range(1):     
    print(prices)
for e in range(1):     
    print(area)
for e in range(1):     
    print(rooms)


#Anzahl der Zeilen in der Liste Printen um zu übverprüfen ob alle die gleiche Länge haben
#Zum Zeitpunkt 13.12.2022 gibt es bei den Listen id,prices, area 20 Einträge
#Eine Liste hat nur 19 Einträge und zwar die Liste Rooms. 
#Ein Vergleich mit dem Quelltext der Website zeigt, dass einmal die Zimmeranzahl nicht angegeben ist
#wichtig! um das ganze Programm zu automatsieren müssen aber die Liste gleich lang sein
print ("Anzahl der Listenelemente ID:", len(id))
print ("Anzahl der Listenelemente Preise:", len(prices))
print ("Anzahl der Listenelemente Fläche:", len(area))
print ("Anzahl der Listenelemente Zimmer:", len(rooms))

['28auw54', '274he5l', '26ekx52', '26zkl56', '27drw5w', '28y9q52', '279kt5y', '27a225p', '27sge5l', '27w5v5j', '27jx85v', '27bge5l', '27nhe5l', '28xyz53', '26eq85w', '26pew5r', '28yzs53', '27yc95r', '26faw5f', '272pd5t']
['2.130.000 €', '690.000 €', '698.000 €', '429.000 €', '548.000 €', '490.000 €', '1.700.000 €', '7.500.000 €', '4.490.000 €', '795.000 €', '699.500 €', '1.100.000 €', '1.140.000 €', '549.000 €', '950.000 €', '1.590.000 €', '760.000 €', '344.000 €', '2.950.000 €', '550.000 €']
['906 m²', '323 m²', '180 m²', '97 m²', '90 m²', '80 m²', '186.62 m²', '524.46 m²', '395 m²', '170 m²', '110.9 m²', '184 m²', '154 m²', '135 m²', '286 m²', '147 m²', '138.87 m²', '190.8 m²', '211 m²', '177.1 m²']
['8 Zi.', '4 Zi.', '4 Zi.', '4 Zi.', '4 Zi.', '8 Zi.', '9.5 Zi.', '6 Zi.', '8 Zi.', '4 Zi.', '5 Zi.', '5 Zi.', '4 Zi.', '9 Zi.', '5 Zi.', '5 Zi.', '1 Zi.', '4 Zi.', '8 Zi.']
Anzahl der Listenelemente ID: 20
Anzahl der Listenelemente Preise: 20
Anzahl der Listenelemente Fläche: 20
Anzahl d

In [9]:
#9. Alle Listen auf die gleiche Länge bringen
#beim xpath befehl zieht der Befehl jeden Wert wo etwas drinsteht raus. Wenn nichts drinsteht überspringt er es
#das heißt wenn nichts drinsteht wird der Wert "Nothing" nicht in der Liste gespeichert
#somit haben die Listen immer unetrschiedliche Längen



#Die LÖSUNG dafür:
#am Anfang hat man die IDs ausgelesen. Da es jede ID nur einmal gibt kann man nach den IDs suchen 
#und dann wie oben mit den Unterstrukturen auf das Element gehen und auslesen


id = tree.xpath(".//a/@id")
rooms = [[0 for s in range(1)] for r in range(len(id))]
area= [[0 for s in range(1)] for r in range(len(id))]
prices = tree.xpath('//div[@data-test="price"]/text()')
MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')

#zuerst einfach gesucht mit dem String 28auw54 eienr ID dann geht man wieder in die Unterstrukturen 
#und liest den Text aus wie schon oben gemacht:
#in diesem Beispiel hat die ID 28auw54 am 13.12.2022 keine Zimmeranzahl stehen, also schreibt der Befehl
#nichts in die Variable rooms rein
#hier nur ein EIntrag also keine Liste da nach einem Spezifischen Wert (die einige ID mit)
# ID Name 28auw54 auf der Seite gesucht wurde
# gesucht wurde und nicht wie bei den 
#bisherigen Befehlen die ganze Seite nach allen IDs durchsucht wurde 
rooms= tree.xpath('//a[@id= "28auw54"]/div[2]/div[1]/div[1]/div[3]/text()')

print(rooms)



[]


In [10]:
#10 Das Ganze jetzt für die Liste mit einer Schleife dabei musste erst der Befehl so umgestaltete werden
# dass man nach dem Variablenwert sucht und nicht nach einem String
#Sonst hätte man jede ID händisch als String in das a[@id= "BeispielidalsString" schreiben müssen
#wurde gelöst! hier der Befehl wenn eine Variable als Suchkriterium einegsetzt wird
#area[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[2]/text()'% id[e])
#der Befehl unerscheidet sich darin dass anstelle der ID ein %s steht und die Variable am Ende hier id[e]
#steht. Dadurch kann man jetzt nach jeder ID auf der Seite suchen und in die Unetrstruktur gehen wo die Fläche 
#steht oder die Raumanzahl. Wie gesagt gibt es nicht immer die Fläche oder die Raumanzahl auf der Website


#um eine dynamische allokierung der Listen rooms und area zu erhalten werden Matrixen erstellt
#mit spalten 2 (range(1)) und Zeilenanzahl (len(id))
#man bezieht sich dann unten nur auf eine Liste indem man z.b. rooms[e][0] macht. Die 0 steht für Spalte 1
#wenn man sich nur in einer Spalte hier Spalte (1) bewegt ist es sozusagen eine Liste. 
rooms = [[0 for s in range(1)] for r in range(len(id))]
area= [[0 for s in range(1)] for r in range(len(id))]


id = tree.xpath(".//a/@id")
prices = tree.xpath('//div[@data-test="price"]/text()')
MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')


#hier die Schleife mit dem Variablenbefehl. Bitte vergleichen mit 9. wo
#der String drin steht ohne Variablenaufruf 
for e in range(len(id)):
    rooms[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[3]/text()'% id[e])
    area[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[2]/text()'% id[e])


for e in range(len(MaxOrt)):
    MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")]


#printen zum Überprüfen
#am 13.12.2022 sind trotz fehlender Informationen in den Zimmern bei den Objekten (Häuser) jeder Listenplatz
#ausgefüllt (auch mit Nothing Werten)
#also es funktioniert

for e in range(1):     
    print(id)
print ("Anzahl der Listenelemente ID:", len(id))

for e in range(1):     
    print(MaxOrt)
print ("Anzahl der Listenelemente Max Ortsabweichung:", len(MaxOrt))
for e in range(1):     
    print(prices)
print ("Anzahl der Listenelemente Preise:", len(prices))
for e in range(1):     
    print(area)
print ("Anzahl der Listenelemente Fläche:", len(area))
for e in range(1):     
    print(rooms)
print ("Anzahl der Listenelemente Zimmer:", len(rooms))

['28auw54', '274he5l', '26ekx52', '26zkl56', '27drw5w', '28y9q52', '279kt5y', '27a225p', '27sge5l', '27w5v5j', '27jx85v', '27bge5l', '27nhe5l', '28xyz53', '26eq85w', '26pew5r', '28yzs53', '27yc95r', '26faw5f', '272pd5t']
Anzahl der Listenelemente ID: 20
['max. 20 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ']
Anzahl der Listenelemente Max Ortsabweichung: 20
['2.130.000 €', '690.000 €', '698.000 €', '429.000 €', '548.000 €', '490.000 €', '1.700.000 €', '7.500.000 €', '4.490.000 €', '795.000 €', '699.500 €', '1.100.000 €', '1.140.000 €', '549.000 €', '950.000 €', '1.590.000 €', '760.000 €', '344.000 €', '2.950.000 €', '550.000 €']
Anzahl der Listenelemente Preise: 20
[[['906 m²']], [['323 m²']], [['180 m²']], [['97 m²']], [['90 m²']], [['80 m²']], 

In [11]:
#11. Das ganze Automatisieren für beliebige URL und Anzahl an Seiten


In [12]:
#11.1 Die URL 
# hier wird wieder eine Matrix allokiert. Diesmal mit Spaltenanzahl len(Ort) und Zeilenanzahl len(Seiten)
#also erhält man eine Matrix mit allen Orten jeweils in eine Spalte Geschrieben 
#für jeden Ort erhält man dann die Url für jede Seite in  den Zeilen

a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
for i in range(len(Ort)):
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        
#printen zum Überprüfen dem Fall werden alle Elemente der Matrix von Oben nach unten und von Links nach rechts
#aufgelistet also zuerst alle Urls zum Ort 0 und Seitenanzahl bis Seitenanzahl len(Seite) dann 
#Ort 1 mit allen Seiten Ort2... etc
for i in range(len(Ort)):
    for j in range(len(Seite)):
        print(a[i][j])


https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=1
https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=2
https://www.immowelt.de/liste/frankfurt/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=1
https://www.immowelt.de/liste/frankfurt/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=2
https://www.immowelt.de/liste/hamburg/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=1
https://www.immowelt.de/liste/hamburg/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=2
https://www.immowelt.de/liste/koeln/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=1
https://www.immowelt.de/liste/koeln/haeuser/kaufen?d=true&efs=

In [13]:
#11.2 Gesamtanzahl an Objekten herausfinden über alle Orte und Seiten 

AnzahlElementebyID=0

a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
for i in range(len(Ort)):
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        page = requests.get(a[i][j])
        tree = html.fromstring(page.content)
        id = tree.xpath(".//a/@id")
        
        AnzahlElementebyID=AnzahlElementebyID+len(id)

#die Gesamtanzahl über alle Websiten (Orte mit Seitenanzahl) der Objekte Häuser ausgeben natürlich mit der 
#vorher Gesamtanzahl an Seiten eingestellten Zahl z.b Hier Seitenanzahl pro Ort = 2
print(AnzahlElementebyID)

200


In [14]:
#12. Das ganze Automatisiert
#funktioniert alles Perfekt mit Fläche und Räume auch wenn manchmal nichts eingetragen ist
# wenn nichts eingetragen ist, ist der Eintrag leer
#csv Datei wird ausgegeben in dem Ordner wo es gespeichert ist
#es fehlt nur noch die maximale Seitenanzahl vom Button. Den auslesen und dann in die Schleifen einbinden
#weil bisher nur händische Eingabe der Seitenanzahl. Die Seitenanzahl ist mal auf zwei Seiten pro Suche 
#hier einegstellt


#ab hier bis zum nächsten hier ist 11.2 vorgeschaltet
AnzahlElementebyID=0

a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
for i in range(len(Ort)):
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        page = requests.get(a[i][j])
        tree = html.fromstring(page.content)
        id = tree.xpath(".//a/@id")
        
        AnzahlElementebyID=AnzahlElementebyID+len(id)
#hier 


#dann die Ausgabe definieren alles wie oben (Punkt 1. bis 11.) bis darauf, 
#dass alles in der Matrix b gespeichert wird 
#die Matrix b hat als Länge der Zeilen die Gesamtanzahl der Objekte über alle Seiten (AnzahlElementebyID)




#[[0 for j in range(Anzahl Spalten)] for i in range(Anzahl Zeilen)]; 
#a[i][j]=a[Zeile][Spalte]=AnzahlElementebyIDert in Zelle der Zeile i und Spalte j
b = [[0 for s in range(7)] for r in range(len(Ort)*AnzahlElementebyID)]
a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
laenge = 0
for i in range(len(Ort)):    
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        page = requests.get(a[i][j])
        tree = html.fromstring(page.content)
        id = tree.xpath(".//a/@id")
        prices = tree.xpath('//div[@data-test="price"]/text()')
        MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')
        for e in range(len(MaxOrt)):
            MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")-1] 
        
        rooms = [[0 for s in range(1)] for r in range(len(id))]
        area= [[0 for s in range(1)] for r in range(len(id))]
        
        for e in range(len(id)):
            rooms[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[3]/text()'% id[e])
            area[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[2]/text()'% id[e])         
        
        
        
        for r in range(len(id)):
             b[laenge+r][0]= id[r] 
             b[laenge+r][1]= MaxOrt[r]     
             b[laenge+r][2]= prices[r]      
             b[laenge+r][3]= Ort[i]
             b[laenge+r][4]=Umkreis
             b[laenge+r][5]= area[r][0]
             b[laenge+r][6]= rooms[r][0]
                    
        
             
        #laenge ist die Aktuelle Anzahl Häusern in der For Schleife 
        # for r in range(len(id)): für die erste Seite fängt es bei b[0+r][Beliebiger Platzhalter] an dann 
        # die zweite Seite ist die Häuseranzahl der ersten Seite laenge=0+len(id) also z.B laenge=20
        # dann fängt für die zweite Seite die b Matrix bei b[20+r][beliebiger Platzhalter] an und geht für
        # r = len(id) also die Anzahl der Häuser ab und addiert diese auf den letzten Wert der vorherigen Seite 
        # siehe  b[laenge+r][Beliebiger Platzhalter]  
        laenge = laenge + len(id) 
        
              
 #jetzt noch in CSV Datei ausgeben:     

data = {
    'ID': [],
    'Ort': [],
    'Umkreis': [],
    'MaxOrt': [],
    'Preis': [],
    'Fläche': [],
    'Zimmer': [],
}

for q in range(AnzahlElementebyID):
    data['ID'].append(b[q][0])
    data['Ort'].append(b[q][3])
    data['Umkreis'].append(b[q][4])
    data['MaxOrt'].append(b[q][1])
    data['Preis'].append(b[q][2])
    data['Fläche'].append(b[q][5])
    data['Zimmer'].append(b[q][6])
    
    
df=pd.DataFrame(data, columns=['ID','Ort','Umkreis','MaxOrt','Preis','Fläche','Zimmer']) 
df.to_csv('Immobilien.csv')
